In [1]:
# pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings("ignore")

from PIL import Image
import cv2

import tensorflow as tf
from tf.keras.utils import load_img
from keras.models import Model
from keras.layers import Conv2d, layer, Dense, Input

import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'tf'

In [3]:
print(tf.__version__)

2.11.0


In [4]:
benign_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign"
normal_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal"
malignant_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant"
benign_dataset = []
normal_dataset = []
malignant_dataset = []

classes_path = [benign_path, normal_path, malignant_path]
classes_dataset = [benign_dataset, normal_dataset, malignant_dataset]

for index, _class in enumerate(classes_path):
    for filename in os.listdir(_class):
        image_path = os.path.join(_class, filename)
        
        if os.path.isfile(image_path) and filename.lower().endswith(').png'):
            mask_path = filename.replace('.png', '_mask.png')
            filename = os.path.join(_class, filename) 
            mask_path = os.path.join(_class, mask_path) 
            classes_dataset[index].append((filename ,mask_path))

In [5]:
color_yellow = '\033[33m' 
color_black = '\033[30m'

length = 0
for _class in classes_dataset:
    print('The length of the {} class is {}'.format(_class[0][0].split('/')[-2], len(_class)))
    length += len(_class)
print(color_yellow + 40*'-')
print(color_black + 'The total length of the dataset is ' + str(length))

The length of the benign class is 437
The length of the normal class is 133
The length of the malignant class is 210
----------------------------------------
The total length of the dataset is 780


In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AttentionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AttentionEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, hidden_size)
        self.attention = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x: input image tensor of shape (batch_size, input_size)
        hidden = torch.tanh(self.linear(x))
        attention_weights = F.softmax(self.attention(hidden), dim=1)
        context_vector = torch.sum(hidden * attention_weights, dim=1)
        return context_vector


class AttentionDecoder(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(hidden_size, num_classes)

    def forward(self, context_vector):
        # context_vector: input context vector tensor of shape (batch_size, hidden_size)
        output = self.linear(context_vector)
        return output


class AttentionClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(AttentionClassifier, self).__init__()
        self.encoder = AttentionEncoder(input_size, hidden_size)
        self.decoder = AttentionDecoder(hidden_size, num_classes)

    def forward(self, x):
        # x: input image tensor of shape (batch_size, input_size)
        context_vector = self.encoder(x)
        output = self.decoder(context_vector)
        return output

class ImageEncoderDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ImageEncoderDecoder, self).__init__()
        self.encoder = AttentionEncoder(input_size, hidden_size)
        self.decoder = AttentionDecoder(hidden_size, num_classes)

    def forward(self, x):
        context_vector = self.encoder(x)
        output = self.decoder(context_vector)
        return output

In [44]:
# Assuming you have prepared your dataset and defined the model as 'model'
num_classes = 10

model = ImageEncoderDecoder(input_size, hidden_size, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Evaluation
    model.eval()
    valid_loss = 0.0
    correct = 0
    
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    
    valid_loss /= len(valid_loader.dataset)
    accuracy = correct / len(valid_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Accuracy: {accuracy:.4f}")

# Save the best model checkpoint
torch.save(model.state_dict(), 'best_model.pt')

# Load the best model checkpoint for testing
model.load_state_dict(torch.load('best_model.pt'))

# Testing
model.eval()
test_loss = 0.0
correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = correct / len(test_loader.dataset)

print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")


NameError: name 'train_loader' is not defined